In [9]:
from coffea.nanoevents import BaseSchema, NanoEventsFactory
import dask_awkward as dak
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import hist
import mplhep as hep
import numba
from coffea.analysis_tools import PackedSelection
import vector
vector.register_awkward()

In [2]:
events = NanoEventsFactory.from_root(
    file="../data/p8_ee_ZH_ecm240/events_101027117.root:events",
    schemaclass=BaseSchema,
    # schemaclass=DelphesSchema, Doesn't work 
    # schemaclass=NanoAODSchema.v7, Doesn't work
    metadata={
        "path":"/eos/experiment/fcc/ee/generation/DelphesEvents/spring2021/IDEA/p8_ee_ZH_ecm240/events_101027117.root",
        "name":"p8_ee_ZH_ecm240",
        "process":"ZH to mumu, ecm=240 GeV"
    }
).events()

In [3]:
from coffea.analysis_tools import PackedSelection

In [4]:
def index_mask(input_array, index_array):
    '''
    This function matches the given attribute of ReconstructedParticles (for example energy) to the particle index (for example Muon or Electron)
    '''
    # input_array = input_array.compute()
    # index_array = index_array.compute()
    if len(input_array) != len(index_array) :
        raise Exception(f'Length of input_array({len(input_array)}) and index_array({len(index_array)}) does not match!')
    counts = len(index_array)
    
    @numba.jit
    def numba_wrap(input_array, index_array, counts):
        output_array = []
        for event_index in range(counts):
            event_mask = index_array[event_index]
            reco_list = input_array[event_index]
            output_array.append([reco_list[i] for i in  event_mask])
        return output_array
    out = ak.Array(numba_wrap(input_array,index_array,counts))
    
    return out

In [ ]:
cut = PackedSelection()
Recon = events['ReconstructedParticles/ReconstructedParticles.energy']
raw_nevents = dak.num(events, axis=0).compute()
print('Raw events : ', raw_nevents)
# useful_events = events[ak.num(Recon) > 0]
cut.add('No cut', ak.all(Recon.compute() > 0, axis=1))
useful_events = ak.mask(events,ak.num(Recon) > 0)
new_Recon = useful_events['ReconstructedParticles/ReconstructedParticles.energy']
reduced_nevents = dak.num(new_Recon, axis=0).compute()
print('Reduced events : ', reduced_nevents)
cut.add('At least one Reco Particle', dak.num(new_Recon).compute() > 0)

Raw events :  100000


In [24]:
cut.names

['No cut', 'At least one Reco Particle']

In [25]:
cut.cutflow(*cut.names).result()

CutflowResult(labels=['initial', 'No cut', 'At least one Reco Particle'], nevonecut=[100000, 100000, 99984], nevcutflow=[100000, 100000, 99984], masksonecut=[array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True])], maskscutflow=[array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True])])